In [44]:
%run authenticate.ipynb
import datetime
import pandas as pd

No rates present in response headers


In [45]:
#authenticate to Strava and get all activities
client = getAuthenticatedClient()
activitiesStrava = client.get_activities()

No rates present in response headers


Some simple functions to export the activities to pandas format and compute useful statistics.

In [46]:
def getActivitiesBike(customIdToBike = True):
    
    dic = {b.id: b.name for b in client.get_athlete().bikes} #bikes are stored with an id in strava.
    
    def gearIdToBike(x, dic): #converts the gear_id to the name of the actual bike I was riding (makes sure there is no None)
        if (x['gear_id'] == None) and (x['start_date_local'] > pd.Timestamp('2024-05-24').tz_localize('UTC')):
            return 'deRosa' #my current main bike
        elif (x['gear_id'] == None) and (x['start_date_local'] <= pd.Timestamp('2024-05-24').tz_localize('UTC')):
            return 'BMC diablo' #my old bike
        else:
            return dic[x['gear_id']]
    
    features = ['name', 'id', 'start_date_local', 'distance', 'gear_id', 'sport_type', 'elapsed_time']
    
    activities = []
    for activity in activitiesStrava:
        activities.append([getattr(activity, f, None) for f in features])
    activities = pd.DataFrame(activities, columns = features)
    
    activities['distance'] = activities['distance'].apply(lambda x: x//1000) #converts to km
    
    activities = activities[(activities['sport_type'] == 'Ride') | (activities['sport_type'] == 'VirtualRide')] #filters out biking activities
    
    if customIdToBike == True: #exchanges gear_id with name of bike. 
        activities['bike'] = activities.apply(lambda x: gearIdToBike(x, dic), axis = 1)
    else:
        activities['bike'] = activities.apply(lambda x: None if x.gear_id == None else dic[x.gear_id], axis = 1)
        
    activities['type'] = activities.sport_type.apply(lambda x: 'Virtual' if x == 'VirtualRide' else 'Outdoor')
    
    activities['date'] = activities.start_date_local.apply(lambda x: x.strftime("%Y-%m-%d"))

    activities['time'] = round(activities['elapsed_time']/3600,1)
    
    return activities[['name', 'date', 'distance', 'bike', 'type', 'time']]



#relevant informations for (my) chains and cassettes in use
chainDeRosa = {
    'Ultegra': ['2024-05-24', '2025-02-18'], #each entry in the list denotes the time the chain was put on (and the other chain removed)
    'Dura Ace': ['2024-12-11']
}
CassetteDeRosa = { #two distinct cassettes, one mounted to indoor trainer, the other on the road bike.
    'Ultegra': {'type': 'Outdoor'},
    '105': {'type': 'Virtual'}
}

def getChainWear():
    activities = getActivitiesBike()
    distinct_dates = sorted(sum(list(chainDeRosa.values()),[]))
    dates = [(start, end) for start,end in zip(distinct_dates, distinct_dates[1:])] + [(distinct_dates[-1], '2222-11-11')]
    
    fullDatesChains = {}
    for c in chainDeRosa.keys():
        fullDatesChains[c] = []
        for (start,end) in dates:
            if start in chainDeRosa[c]:
                fullDatesChains[c].append((start,end))
    for c in fullDatesChains:
        chain_life = sum( activities[(start < activities.date) & (activities.date <= end) & (activities.bike == 'deRosa')].distance.sum() for (start,end) in fullDatesChains[c] )
        print(f"Chain {c}: ", chain_life, " km")

def getCassetteWear():
    activities = getActivitiesBike()
    for cass in CassetteDeRosa:
        wear = activities[(activities.type == CassetteDeRosa[cass]['type']) & (activities.bike == 'deRosa')].distance.sum()
        print(f"Cassette {cass} : {wear}, km")


In [47]:
#returns the last 10 activities and computes the KMs this year
activities = getActivitiesBike(True) #gets the activities
print(activities[:10])
total = activities[activities.date > '2025-01-01'].distance.sum()
print('KMs this year: ', total)

                                   name        date  distance    bike  \
0                Zone 2 Endurance 30min  2025-02-25      24.0  deRosa   
1                  Lactate Threshold #1  2025-02-24      24.0  deRosa   
2  Anaerobic Capacity into Sweetspot #1  2025-02-23      12.0  deRosa   
3                            30/15's #2  2025-02-22      12.0  deRosa   
4                          Endurance #1  2025-02-20      20.0  deRosa   
5                           TT Speed #1  2025-02-19      29.0  deRosa   
6                Zone 2 Endurance 30min  2025-02-18      10.0  deRosa   
7                       Active Recovery  2025-02-17      23.0  deRosa   
8               Endurance with 3 x 1min  2025-02-16      16.0  deRosa   
9                            30/15's #6  2025-02-14      18.0  deRosa   

      type  time  
0  Virtual   1.0  
1  Virtual   1.0  
2  Virtual   0.5  
3  Virtual   0.5  
4  Virtual   1.0  
5  Virtual   1.1  
6  Virtual   1.0  
7  Virtual   0.8  
8  Virtual   0.8  
9  Vir

In [48]:
client.get_athlete()

DetailedAthlete(bound_client=<stravalib.client.Client object at 0x162859540>, id=67722075, city=None, country=None, created_at=datetime.datetime(2020, 9, 4, 20, 38, 39, tzinfo=TzInfo(UTC)), firstname='Moritz', lastname='Venzin', premium=True, profile='avatar/athlete/large.png', profile_medium='avatar/athlete/medium.png', resource_state=3, sex='M', state=None, summit=True, updated_at=datetime.datetime(2025, 2, 9, 20, 40, 32, tzinfo=TzInfo(UTC)), bikes=[SummaryGear(distance=1102561.0, id='b15312545', name='BMC diablo', primary=False, resource_state=2), SummaryGear(distance=406620.0, id='b15308047', name='deRosa', primary=False, resource_state=2)], clubs=[SummaryClub(bound_client=None, id=1235616, name='Transibérica Ultracycling', resource_state=2, activity_types=[ActivityType(root='Handcycle'), ActivityType(root='EBikeRide'), ActivityType(root='VirtualRide'), ActivityType(root='Velomobile'), ActivityType(root='Ride')], city='Bilbao', country='Spain', cover_photo='https://dgalywyr863hv.cl

In [34]:
#shows the wear (KM) for each chain and cassette
getChainWear()
getCassetteWear()

Chain Ultegra:  4521.0  km
Chain Dura Ace:  631.0  km
Cassette Ultegra : 4709.0, km
Cassette 105 : 443.0, km
